 ### 3 (4) возможных способа найти упоминания товара в тексте
 1. Обратиться к названиям товаров + описаниям для того, чтобы выделить паттерны для правил yargy. По этим правилам выделить товары в отзывах. Нюансы: 
 - очень много работы вручную, можно что-то проглядеть или составить слишком много правил 
 - можно увеличить полноту товаров
 2. Можно обратиться к ключевым словам в отзывах, а затем их отфильтровать. Перед этим отфильтровать отзывы от стоп-слов (они не могут быть товарами), убрать пунктуацию. Нюансы:
 - придется фильтровать полученые результаты (возможно с применением правил)/добавлять что-то
 - полнота и точность могут быть низкими
 3. Взять векторные представления текстов, взять названия товаров из metadata. Из названий типа "Seed Scrub" брать обобщающие названия товаров (scrub, shampoo etc) и искать векторную близость (tfidf, bm25) этих названий и текстов. Затем из полученных текстов выделить n-граммы. Нюансы:
 - возможно, корреляция между настоящими названиями и названиями людей товаров будут слабыми (люди могут странно называть товары)
 - сможем объединить все синонимичные упоминания (apple watch, watch)
 4. Обучить на корпусе отзывов word2vec, взять из metadata обобщающие названия (те же shampoo, scrub) и найти наиболее близкие слова. Составить n-граммы (в которых могут быть сложные названия). Нюансы:
 - легко получим синонимичные слова, встречающиеся в похожих контекстах
 - возможны лишние слова

In [56]:
import pandas as pd
import gzip
import json
from tqdm.auto import tqdm
import nltk
nltk.download('averaged_perceptron_tagger')
import re
from collections import Counter
import gensim
from nltk.collocations import *
import spacy
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/assmirnova/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/assmirnova/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [45]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [10]:
df = getDF('Beauty_5.json.gz')

In [12]:
reviews = df['reviewText']
reviews

0         Very oily and creamy. Not at all what I expect...
1         This palette was a decent price and I was look...
2         The texture of this concealer pallet is fantas...
3         I really can't tell what exactly this thing is...
4         It was a little smaller than I expected, but t...
                                ...                        
198497    Just a little dab of this shea butter should b...
198498    This shea butter is completely raw and unrefin...
198499    The skin is the body's largest organ and it ab...
198500    I have very dry elbows and knees.  I have a to...
198501    This is 100% pure Shea Butter. Do not mistake ...
Name: reviewText, Length: 198502, dtype: object

In [13]:
df_meta = getDF('meta_All_Beauty.json.gz')
meta = df_meta[['title', 'description']]
meta

,title,description
0,Loud 'N Clear&trade; Personal Sound Amplifier,[Loud 'N Clear Personal Sound Amplifier allows...
1,No7 Lift &amp; Luminate Triple Action Serum 50...,[No7 Lift & Luminate Triple Action Serum 50ml ...
2,No7 Stay Perfect Foundation Cool Vanilla by No7,[No7 Stay Perfect Foundation now stays perfect...
3,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[]
4,Lacto Calamine Skin Balance Oil control 120 ml...,[Lacto Calamine Skin Balance Daily Nourishing ...
...,...,...
32887,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[]
32888,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[]
32889,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,[]
32890,12 White Feather Shuttlecocks Birdies Badminto...,[Brand new and high quality<br> Enables fast v...


### Реализация способа извлечения названий

In [14]:
all_nouns = []
is_noun = lambda pos: pos[:2] == 'NN'
for title_text in meta['title']:
    clean = re.sub("[^a-zA-Z]+", " ", title_text)
    tokenized = nltk.word_tokenize(clean)
    nouns = [all_nouns.append(word) for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]

Посмотрим на наиболее частые слова в описании: из наиболее содержательных можно выделить такие слова-товары как oil, cream, makeup, brush, gel, spray, soap, lotion, bag, powder, cosmetic, shampoo и так далее. Возьмем некоторые из этих слов при дальнейшей работе с векторизованным корпусом (cream и lotion могут встречаться в одном и том же контексте, поэтому нет смысла брать все эти слова).

In [17]:
c = Counter(all_nouns)
print(c.most_common(100))

[('Hair', 3799), ('Pack', 3464), ('oz', 2797), ('Set', 2224), ('Black', 2082), ('Body', 2032), ('Nail', 2003), ('amp', 1794), ('ml', 1785), ('Oil', 1606), ('Women', 1584), ('Skin', 1528), ('Cream', 1509), ('Color', 1459), ('Oz', 1405), ('Makeup', 1333), ('Brush', 1262), ('Gel', 1262), ('Natural', 1234), ('quot', 1233), ('Ounce', 1160), ('Lip', 1100), ('Spray', 980), ('Kit', 976), ('Face', 958), ('Soap', 956), ('White', 950), ('Eye', 906), ('Beauty', 900), ('Pink', 890), ('g', 838), ('Art', 820), ('Case', 819), ('New', 778), ('Free', 778), ('Lotion', 777), ('Anti', 755), ('Size', 745), ('Blue', 741), ('Bag', 740), ('Powder', 737), ('Fashion', 710), ('Men', 709), ('Care', 705), ('Silver', 699), ('Cosmetic', 666), ('Professional', 658), ('x', 636), ('Travel', 634), ('Bath', 632), ('Long', 631), ('Gold', 625), ('pcs', 624), ('Gift', 619), ('Shampoo', 606), ('Red', 603), ('Polish', 599), ('Hand', 589), ('Facial', 580), ('Mask', 579), ('Crystal', 554), ('Brown', 549), ('Organic', 532), ('Pur

In [18]:
def preprocessing(texts):
    corpus = []
    for text in texts:
        clean_text = re.sub('[^\w ]','',text).lower()
        sent = clean_text.split()
        corpus.append(sent)
    return corpus

In [20]:
corpus = preprocessing(reviews.tolist())
model = gensim.models.Word2Vec(corpus, window=5, sg=0)

In [12]:
model.wv.most_similar('shampoo', topn=15)

[('conditioner', 0.7603300213813782),
 ('shampooconditioner', 0.7103340029716492),
 ('shampoos', 0.678347647190094),
 ('leavein', 0.6327095031738281),
 ('product', 0.6241132616996765),
 ('soap', 0.6173998117446899),
 ('conditoner', 0.6079196929931641),
 ('pantene', 0.6021020412445068),
 ('detangler', 0.5967162251472473),
 ('masque', 0.593921959400177),
 ('cowash', 0.5650419592857361),
 ('cleanser', 0.5531744360923767),
 ('suave', 0.5469362139701843),
 ('wen', 0.5389069318771362),
 ('dove', 0.5372868180274963)]

In [13]:
model.wv.most_similar('product', topn=15)

[('item', 0.7884013056755066),
 ('stuff', 0.7680280208587646),
 ('productit', 0.704575777053833),
 ('serum', 0.6579384207725525),
 ('producti', 0.6500611305236816),
 ('shampoo', 0.6241132020950317),
 ('lotion', 0.6212374567985535),
 ('straightener', 0.5980736017227173),
 ('mascara', 0.5919773578643799),
 ('cleanser', 0.5866108536720276),
 ('tool', 0.5758373737335205),
 ('oneit', 0.5612951517105103),
 ('device', 0.5609607696533203),
 ('moisturizer', 0.5597200989723206),
 ('soap', 0.5572393536567688)]

Можно было бы найти пересечения названий товаров из метадаты и текстов отзывов, но 1) очень долго, тк надо перебирать два списка (они не равны по длине) 2) скорее всего это не принесет пользы, тк мы уже взяли из названий сущ-е, возможно из n-грамм образуются названия

In [ ]:
# intersections = []
# all_titles = preprocessing(meta['title'].tolist())
# for title in tqdm(all_titles):
#     for text in corpus:
#         inter = set.intersection(set(title), set(text))
#         if inter:
#             intersections.append(' '.join(list(inter)))

Можно заметить, что в тех же контекстах что и product встречаются сами названия продуктов. Возьмем слова, полученные моделью от слова product

### Выделение n-грамм

будем собирать n-граммы по полученным названиям продуктов в отзывах

In [21]:
words = ['item', 'stuff', 'serum', 'shampoo', 'lotion', 'straightener', 'mascara', 'cleanser', 'tool', 'device',
         'moisturizer', 'soap']

In [78]:
def full_preprocessing(text):
    clean_text = re.sub('[^\w ]','',text).lower()
    tokens = word_tokenize(clean_text)
    tokens_without_sw = [word for word in clean_text.split() if not word in stopwords]
    return tokens_without_sw

In [87]:
all_tokens = []
for text in tqdm(reviews):
    tokens = full_preprocessing(text)
    all_tokens.append(' '.join(tokens))

  0%|          | 0/198502 [00:00<?, ?it/s]

In [88]:
all_tokens = ' '.join(all_tokens).split()

In [84]:
def ngram_filter(word, n):
    all_ngrams = ngrams(all_tokens, n)
    filtered_ngrams = [x for x in all_ngrams if word in x]
    return filtered_ngrams

Биграммы с нужными нам словами

In [101]:
all_bigrams = {word : [] for word in words}
for word in words:
    bigrams = ngram_filter(word, 2)
    all_bigrams[word].append(bigrams)

In [187]:
for word in words:
    print('---')
    print('СЛОВО', word)
    print('---')
    for bg in all_bigrams[word]:
        for item in bg[:20]:
            print(' '.join(item))

---
СЛОВО item
---
quality item
item installed
negative item
item guess
sized item
item oily
brand item
item since
used item
item father
sponge item
item wish
shelf item
item looking
unhappy item
item oredered
bought item
item really
recommend item
item cetaphil
---
СЛОВО stuff
---
good stuff
stuff 2
campers stuff
stuff great
dried stuff
stuff makes
thick stuff
stuff pumps
amazing stuff
stuff gotten
like stuff
stuff say
good stuff
stuff seasons
trythis stuff
stuff awful
preference stuff
stuff awful
cleanser stuff
stuff lasts
---
СЛОВО serum
---
antifrizz serum
serum spray
hair serum
serum style
regimen serum
serum toner
retinol serum
serum see
c serum
serum timeless
eye serum
serum lol
new serum
serum moisturizer
rx serum
serum hydroquinone
dmae serum
serum also
repair serum
serum exfoliating
---
СЛОВО shampoo
---
disappointed shampoo
shampoo conditioner
thinnest shampoo
shampoo ive
oz shampoo
shampoo used
organics shampoo
shampoo conditioner
biotin shampoo
shampoo shampoo
shampoo hars

Триграммы с нужными нам словами

In [130]:
all_threegrams = {word : [] for word in words}
for word in words:
    threegrams = ngram_filter(word, 3)
    all_threegrams[word].append(threegrams)

In [188]:
for word in words:
    print('---')
    print('СЛОВО', word)
    print('---')
    for bg in all_threegrams[word]:
        for item in bg[:20]:
            print(' '.join(item))

---
СЛОВО item
---
wellhigh quality item
quality item installed
item installed thing
anything negative item
negative item guess
item guess working
trave sized item
sized item oily
item oily formula
store brand item
brand item since
item since price
never used item
used item father
item father used
soft sponge item
sponge item wish
item wish list
stuff shelf item
shelf item looking
---
СЛОВО stuff
---
money good stuff
good stuff 2
stuff 2 thumbs
happy campers stuff
campers stuff great
stuff great first
thicker dried stuff
dried stuff makes
stuff makes hair
even thick stuff
thick stuff pumps
stuff pumps smoothly
years amazing stuff
amazing stuff gotten
stuff gotten hands
home like stuff
like stuff say
stuff say least
sweet good stuff
good stuff seasons
---
СЛОВО serum
---
potent antifrizz serum
antifrizz serum spray
serum spray strong
mousse hair serum
hair serum style
serum style hair
care regimen serum
regimen serum toner
serum toner moisturizer
roc retinol serum
retinol serum see
seru

Ранжирование n-грамм. Я использовала likelihood ratio, PMI и jaccard score. Стоп-слова я исключила, чтобы они не попадали в n-граммы. Частотный фильтр я взяла 15.

In [212]:
corpus_new = preprocessing(reviews.tolist())

In [215]:
clean_corpus = []
for text in corpus_new:
    clean_text = [word for word in text if not word in stopwords]
    text = ' '.join(clean_text)
    clean_corpus.append(text)

In [226]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(' '.join(clean_corpus).split())
finder.apply_freq_filter(15)

In [227]:
pmi = []
lh = []
jaccard = []
for word in words:
    lemma = lemmatizer.lemmatize(word)
    pmi_ratio = [item for item in finder.nbest(bigram_measures.pmi, 100000000000) if lemma in item][:100]
    pmi.append([' '.join(x) for x in pmi_ratio])
    lh_ratio = [item for item in finder.nbest(bigram_measures.likelihood_ratio, 100000000000) if lemma in item][:100]
    lh.append([' '.join(x) for x in lh_ratio])
    jd_ratio = [item for item in finder.nbest(bigram_measures.jaccard, 100000000000) if lemma in item][:100]
    jaccard.append([' '.join(x) for x in jd_ratio])

Результаты ранжирующих метрик:

In [228]:
lh

[['received item',
  'purchased item',
  'item arrived',
  'purchase item',
  'recommend item',
  'bought item',
  'item shipped',
  'addon item',
  'item came',
  'item described',
  'item free',
  'return item',
  'great item',
  'quality item',
  'purchasing item',
  'luxury item',
  'whether item',
  'item amazon',
  'item received',
  'ordered item',
  'receive item',
  'particular item',
  'returned item',
  'add item',
  'returning item',
  'wrong item',
  'buy item',
  'item exactly',
  'got item',
  'item description',
  'item hair',
  'item today',
  'hair item',
  'pleased item',
  'buying item',
  'love item',
  'shipping item',
  'happy item',
  'review item',
  'item price',
  'must item',
  'saw item',
  'beauty item',
  'another item',
  'product item',
  'price item',
  'item works',
  'give item',
  'reviews item',
  'item ordered',
  'item love',
  'item purchased',
  'item took',
  'found item',
  'item purchase',
  'item would',
  'item ive',
  'find item',
  'item

In [229]:
pmi

[['addon item',
  'luxury item',
  'item shipped',
  'returning item',
  'item arrived',
  'item described',
  'returned item',
  'return item',
  'received item',
  'purchased item',
  'receive item',
  'whether item',
  'purchasing item',
  'purchase item',
  'item description',
  'particular item',
  'wrong item',
  'item came',
  'item today',
  'item free',
  'bought item',
  'ordered item',
  'item received',
  'item exactly',
  'add item',
  'quality item',
  'recommend item',
  'shipping item',
  'item amazon',
  'pleased item',
  'saw item',
  'must item',
  'buying item',
  'beauty item',
  'review item',
  'item ordered',
  'happy item',
  'buy item',
  'got item',
  'item took',
  'another item',
  'reviews item',
  'item purchased',
  'great item',
  'give item',
  'item purchase',
  'item price',
  'found item',
  'price item',
  'item works',
  'item thought',
  'item seems',
  'love item',
  'item however',
  'find item',
  'item could',
  'item got',
  'item made',
  '

In [230]:
jaccard

[['item arrived',
  'received item',
  'purchased item',
  'purchase item',
  'item shipped',
  'item described',
  'addon item',
  'item came',
  'return item',
  'bought item',
  'purchasing item',
  'whether item',
  'item free',
  'recommend item',
  'luxury item',
  'quality item',
  'item received',
  'ordered item',
  'receive item',
  'particular item',
  'item amazon',
  'wrong item',
  'returned item',
  'add item',
  'returning item',
  'item exactly',
  'item description',
  'item today',
  'pleased item',
  'shipping item',
  'buying item',
  'buy item',
  'got item',
  'must item',
  'review item',
  'great item',
  'saw item',
  'happy item',
  'beauty item',
  'another item',
  'item ordered',
  'reviews item',
  'item price',
  'item took',
  'item purchased',
  'give item',
  'price item',
  'item works',
  'love item',
  'found item',
  'item purchase',
  'item thought',
  'find item',
  'item however',
  'item love',
  'item seems',
  'item could',
  'item ive',
  '

Тк по всем метрикам результаты вышли похожими, можно выделить их пересечения как итоговый вариант. Посмотрим на результаты триграмм, а затем выведем итоговые результаты.

In [231]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(' '.join(clean_corpus).split())
finder.apply_freq_filter(15)

In [233]:
pmi_3 = []
lh_3 = []
jaccard_3 = []
for word in words:
    lemma = lemmatizer.lemmatize(word)
    pmi_ratio = [item for item in finder.nbest(trigram_measures.pmi, 100000000000) if lemma in item][:100]
    pmi_3.append([' '.join(x) for x in pmi_ratio])
    lh_ratio = [item for item in finder.nbest(trigram_measures.likelihood_ratio, 100000000000) if lemma in item][:100]
    lh_3.append([' '.join(x) for x in lh_ratio])
    jd_ratio = [item for item in finder.nbest(trigram_measures.jaccard, 100000000000) if lemma in item][:100]
    jaccard_3.append([' '.join(x) for x in jd_ratio])

In [234]:
pmi_3

[['item free discount',
  'matter whether item',
  'whether item free',
  'item arrived quickly',
  'received item free',
  'item arrived time',
  'highly recommend item',
  'would recommend item'],
 ['stuff works wonders',
  'absolutely love stuff',
  'stuff smells amazing',
  'best stuff ever',
  'stuff highly recommend',
  'stuff lasts long',
  'highly recommend stuff',
  'stuff works great',
  'stuff goes long',
  'stuff smells great',
  'give stuff try',
  'stuff really works',
  'stuff smells good',
  'hair loves stuff',
  'long way stuff',
  'started using stuff',
  'stuff works well',
  'love stuff smells',
  'dont know stuff',
  'stuff little goes',
  'love stuff makes',
  'stuff dont know',
  'stuff works better',
  'using stuff years',
  'pretty good stuff',
  'stuff makes hair',
  'stuff makes face',
  'stuff smells like',
  'sensitive skin stuff',
  'love love stuff',
  'stuff pretty good',
  'love stuff works',
  'stuff makes skin',
  'stuff ive using',
  'love stuff ive'

In [235]:
lh_3

[['highly recommend item',
  'would recommend item',
  'item arrived quickly',
  'item arrived time',
  'received item free',
  'item free discount',
  'matter whether item',
  'whether item free'],
 ['stuff highly recommend',
  'highly recommend stuff',
  'sensitive skin stuff',
  'stuff works well',
  'stuff ive used',
  'dont know stuff',
  'stuff dont know',
  'stuff goes long',
  'long way stuff',
  'stuff works great',
  'started using stuff',
  'ive using stuff',
  'stuff ive using',
  'absolutely love stuff',
  'really like stuff',
  'stuff little goes',
  'love love stuff',
  'stuff smells great',
  'stuff smells like',
  'stuff smells good',
  'pretty good stuff',
  'stuff makes hair',
  'love stuff love',
  'stuff pretty good',
  'stuff lasts long',
  'stuff makes skin',
  'really love stuff',
  'love stuff works',
  'love stuff smells',
  'really good stuff',
  'stuff works wonders',
  'stuff really good',
  'love stuff makes',
  'love stuff great',
  'love stuff really',
 

In [236]:
jaccard_3

[['matter whether item',
  'item free discount',
  'whether item free',
  'item arrived quickly',
  'received item free',
  'highly recommend item',
  'would recommend item',
  'item arrived time'],
 ['stuff works great',
  'absolutely love stuff',
  'stuff really works',
  'stuff smells amazing',
  'stuff works wonders',
  'stuff smells great',
  'stuff works well',
  'stuff highly recommend',
  'stuff smells good',
  'best stuff ever',
  'highly recommend stuff',
  'long way stuff',
  'stuff goes long',
  'love love stuff',
  'love stuff smells',
  'really like stuff',
  'love stuff use',
  'love stuff makes',
  'give stuff try',
  'stuff makes hair',
  'dont know stuff',
  'stuff lasts long',
  'really love stuff',
  'love stuff great',
  'stuff ive using',
  'love stuff ive',
  'stuff dont know',
  'love stuff works',
  'started using stuff',
  'stuff smells like',
  'stuff little goes',
  'stuff works better',
  'pretty good stuff',
  'really good stuff',
  'ive using stuff',
  's

финальный результат для биграмм и триграмм:

In [239]:
final_bigrams = []
for item in range(0, len(pmi)):
    inter = set(pmi[item]).intersection(lh[item], jaccard[item])
    final_bigrams.append(inter)

In [243]:
for num, bigrams in enumerate(final_bigrams):
    print('биграммы для слова', words[num], ':', list(bigrams)[:5])

биграммы для слова item : ['item im', 'particular item', 'nice item', 'item would', 'item purchased']
биграммы для слова stuff : ['white stuff', 'stuff leaves', 'stuff ever', 'stuff keeps', 'best stuff']
биграммы для слова serum : ['guardian serum', 'serum smells', 'frizz serum', 'retinol serum', 'serum add']
биграммы для слова shampoo : ['mud shampoo', 'volumizing shampoo', 'matching shampoo', 'conditioning shampoo', 'dry shampoo']
биграммы для слова lotion : ['apply lotion', 'putting lotion', 'vaseline lotion', 'moisture lotion', 'drying lotion']
биграммы для слова straightener : ['love straightener', 'dryer straightener', 'straightener worked', 'chi straightener', 'iron straightener']
биграммы для слова mascara : ['mascara black', 'wearing mascara', 'mascara ever', 'mascara eyelashes', 'mascara wand']
биграммы для слова cleanser : ['cleanser leaves', 'renewal cleanser', 'another cleanser', 'cleanser scrub', 'cleanser cleanser']
биграммы для слова tool : ['tool would', 'using tool', 

In [244]:
final_trigrams = []
for item in range(0, len(pmi_3)):
    inter = set(pmi_3[item]).intersection(lh_3[item], jaccard_3[item])
    final_trigrams.append(inter)

In [245]:
for num, trigrams in enumerate(final_trigrams):
    print('триграммы для слова', words[num], ':', list(trigrams)[:5])

триграммы для слова item : ['matter whether item', 'highly recommend item', 'item arrived quickly', 'received item free', 'item free discount']
триграммы для слова stuff : ['stuff works well', 'ive using stuff', 'stuff made hair', 'great stuff love', 'good stuff like']
триграммы для слова serum : ['serum works well', 'ive using serum', 'serum twice day', 'serum 2 weeks', 'guardian serum growth']
триграммы для слова shampoo : ['1 shampoo conditioner', 'body wash shampoo', 'nourishing shampoo conditioner', 'shampoo conditioner leave', 'highly recommend shampoo']
триграммы для слова lotion : ['would recommend lotion', 'lotion every day', 'highly recommend lotion', 'lotion pure oat', 'foaming wrap lotion']
триграммы для слова straightener : ['ceramic hair straightener', 'straightener heats quickly', 'straightener pearl infused', 'hair straightener pearl']
триграммы для слова mascara : ['mascara ever used', 'really like mascara', 'mascara ive ever', 'eye liner mascara', 'ive using mascara']

Можно заметить, что слова stuff, product, которые являются референтами к продуктам, используются в основном для оценки товара (highly recommend item, great stuff love), в то время как слова, которые более конктретно отражают продукт типа shampoo, oil используются с их описательными характеристиками (olive oil, body wash shampoo).